In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import os

In [ ]:
%run notebook_setup

# Load data in from Google Drive

from google.colab import drive
drive.mount('/content/drive')

# Transit fitting

*exoplanet* includes methods for computing the light curves transiting planets.
In its simplest form this can be used to evaluate a light curve like you would do with [batman](https://astro.uchicago.edu/~kreidberg/batman/), for example:

In [ ]:
import os
HOME = os.environ['HOME']
working_dir = os.path.join(HOME, 'path/to/notebooks/')
os.chdir(working_dir)

In [ ]:
from astropy import units
from exomast_api import exoMAST_API
from time import time

import exoplanet as xo
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pymc3 as pm
import starry
import theano.tensor as tt
import theano

starry.config.quiet = True

In [ ]:
import joblib
import pandas as pd

from tqdm import tqdm_notebook as tqdm

In [ ]:
from arctor import Arctor, info_message
from arctor.utils import instantiate_arctor, create_raw_lc_stddev, instantiate_star_planet_system

In [ ]:
plot_verbose = False
save_now = False
planet_name = 'planet_name'
file_type = 'flt.fits'

HOME = os.environ['HOME']
base_dir = os.path.join(HOME, 'Research', 'Planets', planet_name)
data_dir = os.path.join(base_dir, 'path', 'to', 'FLT', 'files')
data_dir = os.path.join(data_dir, 'HST', 'FLTs')
working_dir = os.path.join(base_dir, 'path', 'to', 'savefiles')

In [ ]:
planet = instantiate_arctor(planet_name, data_dir, working_dir, file_type)

In [ ]:
joblib_filename = 'name_of_savefile_100x100_finescale.joblib.save'
joblib_filename = f'{working_dir}/{joblib_filename}'
planet.load_dict(joblib_filename)

In [ ]:
planet.photometry_df.columns

In [ ]:
if not hasattr(planet, 'gaussian_centers'):
    planet.load_dict(joblib_filename)
    planet.clean_cosmic_rays()
    planet.calibration_trace_location()
    planet.identify_trace_direction()
    planet.simple_phots()
    planet.center_all_traces()
    planet.fit_trace_slopes()
    planet.compute_sky_background(subpixels=32)
    planet.compute_columnwise_sky_background()

# Run Multi-Phot

In [ ]:
if not hasattr(planet, 'photometry_df'):
    # Set up the list of aperture widths and heights to search
    min_aper_width = 1
    max_aper_width = 100
    min_aper_height = 1
    max_aper_height = 100

    aper_widths = np.arange(min_aper_width, max_aper_width + 2, 5)
    aper_heights = np.arange(min_aper_height, max_aper_height + 2, 5)
    
    planet.do_multi_phot(aper_widths, aper_heights)

# Determine the 'best' photometry SNR

In [ ]:
med_photometry_df = np.median(planet.photometry_df, axis=0)
planet.normed_photometry_df = planet.photometry_df / med_photometry_df
planet.normed_uncertainty_df = np.sqrt(planet.photometry_df) / med_photometry_df

In [ ]:
planet_fine_photometry_df = planet.photometry_df.copy()

In [ ]:
fine_snr_lightcurves = create_raw_lc_stddev(planet)
fine_min_snr = fine_snr_lightcurves[fine_snr_lightcurves.argmin()]
fine_min_snr_colname = planet.photometry_df.columns[fine_snr_lightcurves.argmin()]
fine_min_snr_flux = planet.normed_photometry_df[fine_min_snr_colname]
fine_min_snr_uncs = planet.normed_uncertainty_df[fine_min_snr_colname]
fine_temp = fine_min_snr_colname.split('_')[-1].split('x')
fine_min_snr_aper_width, fine_min_snr_aper_height = np.int32(fine_temp)

In [ ]:
info_message(f'Fine Aperture Photometry Resulted in {fine_min_snr:0.0f}ppm with '
             f'{fine_min_snr_aper_width}x{fine_min_snr_aper_height} aperture size; '
             f'with median uncertainties of {np.median(fine_min_snr_uncs)*1e6:0.0f} ppm')

# Configure system for PyMC3

In [ ]:
idx_fwd = planet.idx_fwd
idx_rev = planet.idx_rev

In [ ]:
# Compute a limb-darkened light curve using starry
times = planet.times
u = []
flux = planet.normed_photometry_df['aperture_sum_16x51']
yerr = planet.normed_uncertainty_df['aperture_sum_16x51']

plt.errorbar(times[idx_fwd], flux[idx_fwd], yerr[idx_fwd], fmt='o', color="C0")
plt.errorbar(times[idx_rev], flux[idx_rev], yerr[idx_rev], fmt='o', color="C3")
plt.axhline(1.0, ls='--', color='C1')
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.xlim(times.min(), times.max());

But the real power comes from the fact that this is defined as a [Theano operation](http://deeplearning.net/software/theano/extending/extending_theano.html) so it can be combined with PyMC3 to do transit inference using Hamiltonian Monte Carlo.

## The transit model in PyMC3

In this section, we will construct a simple transit fit model using *PyMC3* and then we will fit a two planet model to simulated data.
To start, let's randomly sample some periods and phases and then define the time sampling:

In [ ]:
np.random.seed(42)

time_med = np.median(times)
med_t_diff = np.median(np.diff(times))

Then, define the parameters.
In this simple model, we'll just fit for the limb darkening parameters of the star, and the period, phase, impact parameter, and radius ratio of the planets (note: this is already 10 parameters and running MCMC to convergence using [emcee](https://emcee.readthedocs.io) would probably take at least an hour).
For the limb darkening, we'll use a quadratic law as parameterized by [Kipping (2013)](https://arxiv.org/abs/1308.0009).
This reparameterizations is implemented in *exoplanet* as custom *PyMC3* distribution :class:`exoplanet.distributions.QuadLimbDark`.

In [ ]:
print(f'This instance has {mp.cpu_count()} CPUs')

In [ ]:
import pymc3 as pm
b = 0.66 # Hellier 2011
period = 0.813475  # days # exo.mast.stsci.edu
u = [0]
t0 = time_med
edepth = np.sqrt(1000/1e6)

orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0, b=b)
injected_light_curves = xo.LimbDarkLightCurve(u).get_light_curve(orbit=orbit, r=edepth, t=times).eval().flatten()

In [ ]:
plt.errorbar(times, flux * (injected_light_curves+1), yerr, fmt='o')
plt.plot(times, injected_light_curves+1,'o')
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.xlim(times.min(), times.max());

In [ ]:
t0_planet = 55528.3684  # exo.mast.stsci.edu
n_epochs = np.int(np.round(((time_med - t0_planet) / period)-0.5))
n_epochs, ((time_med - t0_planet) / period)
t0_guess = t0_planet + (n_epochs+0.5) * period  # eclipse

# t0s = np.random.normal(t0_guess, 0.1*med_t_diff, size=2)
t0s = t0_guess

# Run 400 MCMCs

In [ ]:
b = 0.66 # Hellier 2011
period = 0.813475  # days # exo.mast.stsci.edu
u = [0]

oot_guess = np.median(np.r_[flux[:2*18], flux[-18:]])
# stellar_variance = np.std(np.r_[flux[:2*18], flux[-18:]])
data = flux * (injected_light_curves+1)# - oot_guess
t0 = t0_guess

edepth = np.sqrt(1000/1e6)
orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0, b=b)
injected_light_curve = xo.LimbDarkLightCurve(u).get_light_curve(orbit=orbit, r=edepth, t=times).eval().flatten()

In [ ]:
planet_info = exoMAST_API('PlanetNameb')

In [ ]:
kwargs = {}

# Stellar parameters
kwargs['star_ydeg'] = 0
kwargs['star_udeg'] = 2
kwargs['star_L'] = 1.0
kwargs['star_inc'] = 90.0
kwargs['star_obl'] = 0.0
kwargs['star_m'] = planet_info.Ms
kwargs['star_r'] = planet_info.Rs
kwargs['star_prot'] = 1.0
kwargs['star_t0'] = 0
kwargs['star_theta0'] = 0.0
kwargs['star_A1'] = 1.0
kwargs['star_A2'] = 0.0

# Planetary parameters
kwargs['planet_B1'] = 1.0
kwargs['planet_ydeg'] = 0
kwargs['planet_udeg'] = 0
kwargs['planet_L']  =  5e-4
kwargs['planet_a'] = 1.0
kwargs['planet_phase_offset'] = 0.
kwargs['planet_inc'] = planet_info.inclination
kwargs['planet_porb'] = planet_info.orbital_period
kwargs['planet_t0'] = 0.0#planet_info.transit_time
kwargs['planet_obl'] = 0.0
kwargs['planet_m'] = 0.0  # planet_info.Mp
kwargs['planet_r'] = planet_info.Rp
kwargs['planet_ecc'] = planet_info.eccentricity
kwargs['planet_w'] = planet_info.omega
kwargs['planet_Omega'] = 0.0
kwargs['planet_theta0'] = 0.0

sys = instantiate_star_planet_system(**kwargs)
# map_soln, mcmc_trace = run_pymc3(times, flux, yerr, tune=250, draws=500, target_accept=0.9, **kwargs)

In [ ]:
artificial_times = np.linspace(times_mod.min(), times_mod.max(), 300)
sys.show(t=artificial_times, window_pad=1, figsize=(5, 5))

In [ ]:
n_epochs = np.int(np.round(np.median(((times - planet_info.transit_time)/planet_info.orbital_period) - 0.5)))
times_mod = times - planet_info.transit_time - n_epochs*planet_info.orbital_period

In [ ]:
artificial_times = times_mod
flux_true = sys.flux(artificial_times).eval()
ferr = 1e-4
flux = flux_true + yerr * np.random.randn(len(artificial_times))
plt.errorbar(artificial_times, flux, yerr, fmt="ko", alpha=0.3, ms=3, zorder=2)
plt.plot(artificial_times, flux_true, 'o');

In [ ]:
import pymc3 as pm
b = 0.66 # Hellier 2011
period = 0.813475  # days # exo.mast.stsci.edu
u = [0]

# oot_guess = np.median(np.r_[flux[:2*18], flux[-18:]])
# stellar_variance = np.std(np.r_[flux[:2*18], flux[-18:]])
data = flux * (injected_light_curves+1)# - oot_guess
t0 = t0_guess

def run_pymc3(times, flux, ferr, xcenters, tune=3000, draws=3000, target_accept=0.9, do_mcmc_now=False, **kwargs):
    """
        KWARGS:
             # Stellar parameters
             star_ydeg=0, star_udeg=2, star_L=1.0, star_inc=90.0, 
             star_obl=0.0, star_m=1.0, star_r=1.0, star_prot=1.0, 
             star_t0=0, star_theta0=0.0, star_A1=0.4, star_A2=0.2, 
             # Planetary parameters
             planet_B1=1.0, planet_ydeg=1, planet_udeg=0, planet_L = 1.0, 
             planet_a=1.0, planet_phase_offset=0., planet_inc=90.0, planet_porb=1.0,
             planet_t0=0.0, planet_obl=0.0, planet_m=0.0, planet_r=0.1, planet_ecc=0.0,
             planet_w=90.0, planet_Omega=0.0, planet_theta0=0.0):
    """
    with pm.Model() as model:
        # planet_phase_offset = pm.Normal("planet_phase_offset", 0.0, 30.0, testval=0.11)
        # log_L = pm.Normal("log_L", -4.0, 2.0, testval=-3.91)
        log_L = pm.Uniform("log_L", -np.inf, 0)
        mean = pm.Normal("mean", mu=0.0, sd=1.0)
            
        # kwargs['planet_phase_offset'] = planet_phase_offset
        kwargs['planet_L'] = 10**log_L
        
        sys = instantiate_star_planet_system(**kwargs)
        light_curve = sys.flux(times)
        
        slope = pm.Uniform("slope", lower=-0.1, upper=0.1)
#         slope_xc = pm.Uniform("slope_xcenter", lower=-0.1, upper=0.1)
        
        line = means + slope * times + slope_xc * xcenters
        
        model_ = light_curve + line
        
        flux_model = pm.Deterministic("flux_model", model_)
        pm.Normal("obs", flux_model, sd=ferr, observed=flux)
        
        start0 = time()
        map_soln = xo.optimize()
        print(f'[INFO] MAP Solution took {time() - start0} seconds')
        
        if do_mcmc_now:
            start1 = time()
            trace = pm.sample(
                tune=tune,
                draws=draws,
                start=map_soln,
                chains=mp.cpu_count(),
                cores=mp.cpu_count(),
                step=xo.get_dense_nuts_step(target_accept=target_accept),
            )
            print(f'[INFO] MCMC took {time() - start1} seconds')
        else:
            trace = None
        print(f'[INFO] Full Process took {time() - start0} seconds')
        
    return map_soln, trace

In [ ]:
kwargs['planet_L'] = 

In [ ]:
sys = instantiate_star_planet_system(**kwargs)

In [ ]:
fine_snr_flux = planet.normed_photometry_df
fine_snr_uncs = planet.normed_uncertainty_df

n_columns = len(fine_snr_flux.columns)

aper_sum_columns = planet.normed_photometry_df.drop(
    ['xcenter', 'ycenter'], axis=1).columns

xcenters = planet.photometry_df['xcenter']
xcenters_mod = xcenters - np.median(xcenters)

n_epochs = np.int(np.round(np.median(((times - planet_info.transit_time)/planet_info.orbital_period) - 0.5)))
times_mod = times - planet_info.transit_time - n_epochs*planet_info.orbital_period


do_mcmc_now = False
save_as_you_go = False

# varnames = ["edepth", "slope_xcenter", "mean", "slope"]
varnames = ['log_L', 'mean', 'slope', 'slope_xcenter']
fine_grain_mcmcs_w_xcenterfit = {}
for colname in aper_sum_columns:  # [fine_min_snr_colname]:  # 
    start = time()
    info_message(f'Working on {colname} for Trace MCMCs')
    flux = fine_snr_flux[colname] * (injected_light_curve + 1.0)
    ferr = fine_snr_uncs[colname]
    
    map_soln, trace  = run_pymc3(times, flux, ferr, xcenters, do_mcmc_now=do_mcmc_now,
                                 tune=3000, draws=3000, target_accept=0.9, **kwargs)
    fine_grain_mcmcs_w_xcenterfit[colname] = {}
    fine_grain_mcmcs_w_xcenterfit[colname]['trace'] = trace
    fine_grain_mcmcs_w_xcenterfit[colname]['map_soln'] = map_soln
    
    if save_as_you_go:
        info_message(f'Completed {colname} for Trace MCMCs took {time()-start}')
        filename = f'{planet_name}_fine_grain_photometry_100x100_XXXppm_100MAPs.joblib.save'
        filename = os.path.join(working_dir, filename)

        info_message(f'Saving MCMCs to {filename}')
        joblib.dump(fine_grain_mcmcs_w_xcenterfit, filename)
    if trace is not None:
        print(pm.summary(trace, var_names=varnames))

In [ ]:
%%timeit
filename = f'{planet_name}_fine_grain_photometry_100x100_XXXppm_100MAPs.joblib.save'
filename = os.path.join(working_dir, filename)

info_message(f'Saving MCMCs to {filename}')
joblib.dump(fine_grain_mcmcs_w_xcenterfit, filename)

In [ ]:
varnames = ['log_L', 'mean', 'slope', 'slope_xcenter']
pm.summary(trace, varnames=varnames)

In [ ]:
import corner
samples = pm.trace_to_dataframe(trace, varnames=varnames)
corner.corner(np.array(samples), truths=[-3, 10, 0., 0.], labels=varnames);

Now we can plot the simulated data and the maximum a posteriori model to make sure that our initialization looks ok.

## Sampling

Now, let's sample from the posterior defined by this model.
As usual, there are strong covariances between some of the parameters so we'll use :func:`exoplanet.get_dense_nuts_step`.

After sampling, it's important that we assess convergence.
We can do that using the `pymc3.summary` function:

In [ ]:
aper_sum_columns = planet.normed_photometry_df.drop(
    ['xcenter', 'ycenter'], axis=1).columns
aper_sum_columns

In [ ]:
varnames = ["edepth", "mean", "slope", "slope_xcenter"]

In [ ]:
trace_ = fine_grain_mcmcs_w_xcenterfit[colname]['trace']
dir(trace_)
trace_.report._gelman_rubin.values()
dir(trace_.report)
trace_.report._run_convergence_checks??

In [ ]:
# fine_grain_mcmcs_w_xcenterfit[colname]['map_soln']
edepths = []
means = []
slopes = []
slopes_xcenter = []

edepths_unc = []
means_unc = []
slopes_unc = []
slopes_xcenter_unc = []

mesh_widths = []
mesh_heights = []

for colname in tqdm(aper_sum_columns):
    aper_width_, aper_height_ = np.int32(colname.split('_')[-1].split('x'))
    mesh_widths.append(aper_width_)
    mesh_heights.append(aper_height_)

    # Load Summary from Colname
    summary_df = pm.summary(fine_grain_mcmcs_w_xcenterfit[colname]['trace'], varnames=varnames)

    # Store mean values
    edepths.append(summary_df['mean'].loc['edepth'])
    means.append(summary_df['mean'].loc['mean'])
    slopes.append(summary_df['mean'].loc['slope'])
    slopes_xcenter.append(summary_df['mean'].loc['slope_xcenter'])

    # Store uncertainties
    edepths_unc.append(summary_df['sd'].loc['edepth'])
    means_unc.append(summary_df['sd'].loc['mean'])
    slopes_unc.append(summary_df['sd'].loc['slope'])
    slopes_xcenter_unc.append(summary_df['sd'].loc['slope_xcenter'])

In [ ]:
df_columns = ["aper_width", "aper_height", "edepth", "mean", "slope", "slope_xcenter"]
means_df = pd.DataFrame(np.transpose([mesh_widths, mesh_heights, edepths, means, slopes, slopes_xcenter]), 
                        columns=df_columns)
uncs_df = pd.DataFrame(np.transpose([mesh_widths, mesh_heights, edepths_unc, means_unc, slopes_unc, slopes_xcenter_unc]), 
                        columns=df_columns)

In [ ]:
# mesh_widths_sorted = np.argsort(mesh_widths)
# mesh_heights_sorted = np.argsort(mesh_heights)
plt.plot(means_df['aper_width'], (means_df['edepth'] / uncs_df['edepth']), '.')
plt.plot(means_df['aper_height'], (means_df['edepth'] / uncs_df['edepth']),'.')

plt.figure()
plt.plot(means_df['aper_height']*means_df['aper_width'], (means_df['edepth'] / uncs_df['edepth']),'.')

In [ ]:
mesh_widths_sorted = np.argsort(mesh_widths)
mesh_heights_sorted = np.argsort(mesh_heights)

plt.figure()
plt.plot(means_df['aper_width'], means_df['mean'],'.')
plt.plot(means_df['aper_height'], means_df['mean'],'.')

plt.figure()
plt.plot(means_df['aper_height']*means_df['aper_width'], means_df['mean'],'.')

That looks pretty good!
Fitting this without *exoplanet* would have taken a lot more patience.

Now we can also look at the [corner plot](https://corner.readthedocs.io) of some of that parameters of interest:

In [ ]:
import pygtc
import corner

In [ ]:
trace = fine_grain_mcmcs_w_xcenterfit[colname]['trace']

In [ ]:
n_lightcurves_fwd = trace['light_curves_fwd'].shape[0]
light_curves_fwd = trace['light_curves_fwd'].reshape(trace['light_curves_fwd'].shape[:2])#trace['mean_fwd'][:,None]

n_lightcurves_rev = trace['light_curves_rev'].shape[0]
light_curves_rev = trace['light_curves_rev'].reshape(trace['light_curves_rev'].shape[:2])#trace['mean_rev'][:,None]

for k in np.random.choice(np.arange(n_lightcurves_fwd), size=100):
    plt.plot(t[idx_fwd], light_curves_fwd[k], '.', color='C0', alpha=0.25)

for k in np.random.choice(np.arange(n_lightcurves_rev), size=100):
    plt.plot(t[idx_rev], light_curves_rev[k], '.', color='C1', alpha=0.25)

plt.ylim(-1e-5, 1e-6);
plt.xlim(t0_guess - 0.1, t0_guess + 0.1);

In [ ]:
for colname in tqdm(aper_sum_columns):
    trace = fine_grain_mcmcs_w_xcenterfit[colname]['trace']
    
    samples = pm.trace_to_dataframe(trace, varnames=varnames)
    truth = [0.0, 1.0, 1.0, 0.0]
    corner.corner(samples, truths=truth, labels=varnames);

## Phase plots

Like in the radial velocity tutorial (:ref:`rv`), we can make plots of the model predictions for each planet.

In [ ]:
line_fit.shape

In [ ]:
plt.figure()
# Get the posterior median orbital parameters
p = period
# t0 = np.median(trace["t0"])

# Plot the folded data
line_fit =  + trace['slope_xcenter'] + trace['slope']
line_fit = slope * (t-t0_guess) + mean + slope * (xcenters-np.median(xcenters))

plt.errorbar(times[idx_fwd] - t0, (data - line_fit + 1.0)[idx_fwd], yerr=yerr[idx_fwd], fmt=".", color='C0', label="fwd data", zorder=-1000)
plt.errorbar(times[idx_rev] - t0, (data - line_fit + 1.0)[idx_rev], yerr=yerr[idx_rev], fmt=".", color='C3', label="rev data", zorder=-1000)

# Plot the folded model
preds_fwd = trace["light_curves"][:,:,0]
# preds_rev = trace["light_curves"][:,:,0] + trace["mean"][:, None]
pred_fwd = np.median(preds_fwd, axis=0)
# pred_rev = np.median(preds_rev, axis=0)


plt.plot(times - t0, pred_fwd, color="C1", label="model", zorder=10)
# plt.plot(t[idx_rev] - t0, pred_rev, color="C2", label="model", zorder=10)
plt.axhline(1.0, ls='--', color='k')

# Annotate the plot with the planet's period
txt = f"Eclipse Depth = {np.mean(trace['edepth']*1e6):.0f}"
txt += f" +/- {np.std(trace['edepth']*1e6):.0f} ppm"

plt.annotate(
    txt,
    (0, 0),
    xycoords="axes fraction",
    xytext=(5, 5),
    textcoords="offset points",
    ha="left",
    va="bottom",
    fontsize=12,
)

add_traces = False
if add_traces:
    n_traces = 1000
    
    idx_rand = np.random.choice(np.arange(preds_fwd.shape[0]), size=n_traces, replace=False)
    for pred_ in preds_fwd[idx_rand]:
        plt.plot(times - t0, pred_, color="grey", alpha=0.5, zorder=0)

    # idx_rand = np.random.choice(np.arange(preds_fwd.shape[0]), size=n_traces, replace=False)
    # for pred_ in preds_rev[idx_rand]:
    #     plt.plot(t[idx_rev] - t0, pred_, color="grey", alpha=0.5, zorder=0)

plt.legend(fontsize=10, loc=4)
plt.xlim((times - t0).min(), (times - t0).max())
plt.xlabel("Time Since Eclipse [days]")
plt.ylabel("Relative Flux")
plt.title("PlanetName UVIS Eclipse");

## Citations

As described in the :ref:`citation` tutorial, we can use :func:`exoplanet.citations.get_citations_for_model` to construct an acknowledgement and BibTeX listing that includes the relevant citations for this model.
This is especially important here because we have used quite a few model components that should be cited.

In [ ]:
with model:
    txt, bib = xo.citations.get_citations_for_model()
print(txt)

In [ ]:
print("\n".join(bib.splitlines()[:10]) + "\n...")